In [ ]:
#const BACKEND = :CUDA
#Pkg.activate(string(BACKEND, "Env"))

In [1]:
using Pkg
Pkg.status()
Pkg.add("HDF5")
Pkg.add("CUDA")
Pkg.add("JupyterFormatter")

Status `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [052768ef] CUDA v4.3.0
  [f67ccb44] HDF5 v0.16.14
  [b8b539d8] JupyterFormatter v0.1.0
  [91a5bcdd] Plots v1.38.12
  [9ff05d80] TickTock v1.2.0


   Resolving package versions...
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Project.toml`
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Project.toml`
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Project.toml`
  No Changes to `C:\Users\Mishka\Documents\dykes_julia\dykes_julia\Manifest.toml`


In [2]:
using CUDA
using Printf
using BenchmarkTools

In [3]:
CUDA.versioninfo()

CUDA runtime 12.1, artifact installation
CUDA driver 12.0
NVIDIA driver 528.33.0

CUDA libraries: 
- CUBLAS: 12.1.3
- CURAND: 10.3.2
- CUFFT: 11.0.2
- CUSOLVER: 11.4.5
- CUSPARSE: 12.1.0
- CUPTI: 18.0.0
- NVML: 12.0.0+528.33

Julia packages: 
- CUDA.jl: 4.3.0
- CUDA_Driver_jll: 0.5.0+1
- CUDA_Runtime_jll: 0.6.0+0
- CUDA_Runtime_Discovery: 0.2.2

Toolchain:
- Julia: 1.9.0
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: NVIDIA GeForce RTX 3070 Laptop GPU (sm_86, 7.091 GiB / 8.000 GiB available)


In [4]:
collect(devices())

1-element Vector{CuDevice}:
 CuDevice(0): NVIDIA GeForce RTX 3070 Laptop GPU

In [5]:
function kernel()
    dev = Ref{Cint}()
    CUDA.cudaGetDevice(dev)
    @cuprintln("Running on device $(dev[])")
    return
end

kernel (generic function with 1 method)

In [6]:
kernel()

Running on device 0


In [7]:
function print_gpu_properties()

    for (i, device) in enumerate(CUDA.devices())
        println("*** General properties for device $i ***")
        name = CUDA.name(device)
        println("Device name: $name")
        major = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR)
        minor = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR)
        println("Compute capabilities: $major.$minor")
        clock_rate = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_CLOCK_RATE)
        println("Clock rate: $clock_rate")
        device_overlap = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_GPU_OVERLAP)
        print("Device copy overlap: ")
        println(device_overlap > 0 ? "enabled" : "disabled")
        kernel_exec_timeout =
            CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_KERNEL_EXEC_TIMEOUT)
        print("Kernel execution timeout: ")
        println(kernel_exec_timeout > 0 ? "enabled" : "disabled")
    end
end

print_gpu_properties (generic function with 1 method)

In [8]:
function read_par(par, ipar)
    par_name_2 = par[ipar]
    ipar_2 = ipar + 1
    return par_name_2, ipar_2
end

read_par (generic function with 1 method)

In [9]:
using HDF5
using Printf

In [10]:
#using JupyterFormatter
#enable_autoformat()

In [11]:
#macro ALL_ARGS()
#    T, T_old, C, wts, px, py, pT, pPh, lam_r_rhoCp, lam_m_rhoCp, L_Cp, T_top, T_bot, dx, dy, dt, pic_amount, nx, ny, npartcl, npartcl0
#end

In [12]:
#macro ALL_PARAMS()
#     T, T_old, C, wts, px, py, pT, pPh, lam_r_rhoCp, lam_m_rhoCp, L_Cp, T_top, T_bot, dx, dy, dt, pic_amount, nx, ny, npartcl, npartcl0
#end

In [13]:
function idc(ix, iy, nx)
    return (iy) * nx + (ix)
end

idc (generic function with 1 method)

In [14]:
function blerp(x1, x2, y1, y2, f11, f12, f21, f22, x, y)
    invDxDy = 1.0 / ((x2 - x1) * (y2 - y1))

    dx1 = x - x1
    dx2 = x2 - x

    dy1 = y - y1
    dy2 = y2 - y

    return invDxDy * (f11 * dx2 * dy2 + f12 * dx2 * dy1 + f21 * dx1 * dy2 + f22 * dx1 * dy1)
end

blerp (generic function with 1 method)

In [15]:
function g2p!(T, T_old, C, wts, px, py, pT, pPh, lam_r_rhoCp, lam_m_rhoCp, L_Cp, T_top, T_bot, dx, dy, dt, pic_amount, nx, ny, npartcl, npartcl0)
    ip = blockIdx().x * blockDim().x + threadIdx().x;

    if ip > (npartcl - 1)
        return nothing
    end

    pxi = px[ip]/ dx;
    pyi = px[ip]/ dy;

    ix1 = min(max(convert(Int64, pxi), 0), nx - 2);
    iy1 = min(max(convert(Int64, pyi), 0), ny - 2);
    
    ix2 = ix1 + 1;
    iy2 = iy1 + 1;
    
    x1 = ix1 * dx
    x2 = ix2 * dx;
    y1 = iy1 * dy;
    y2 = iy2 * dy;
    
    T_pic = blerp(x1, x2, y1, y2, T[idc(ix1, iy1, nx)], T[idc(ix1, iy2, nx)], T[idc(ix2, iy1, nx)], T[idc(ix2, iy2, nx)], px[ip], py[ip]);
    T_flip = pT[ip] + T_pic - blerp(x1, x2, y1, y2, T_old[idc(ix1, iy1, nx)], T_old[idc(ix1, iy2, nx)], T_old[idc(ix2, iy1, nx)], T_old[idc(ix2, iy2,nx)], px[ip], py[ip]);
    pT[ip] = T_pic * pic_amount + T_flip * (1.0 - pic_amount);
    
    return nothing
end

g2p! (generic function with 1 method)

In [16]:
function init_particles_T(pT, T_magma, maxpartcl, npartcl)
    ip = blockIdx().x * blockDim().x + threadIdx().x

    if (ip > (maxpartcl - npartcl - 1))
        return
    end

    pT[ip+npartcl] = T_magma
    return
end

init_particles_T (generic function with 1 method)

In [17]:
#avg += mf_magma(T[idc(ix, iy, nx)]) * vf + mf_rock(T[idc(ix, iy, nx)]) * (1 - vf);
function mf_magma(T)
    t2 = T * T
    t7 = exp(
        0.961026371384066e3 - 0.3590508961e1 * T + 0.4479483398e-2 * t2 -
        0.1866187556e-5 * t2 * T,
    )
    return 0.1e1 / (0.1e1 + t7)
end

#=
double mf_rhyolite(double T) {
  double t2 = T * T;
  double t7 = exp(0.961026371384066e3 - 0.3590508961e1 * T + 0.4479483398e-2 * t2 - 0.1866187556e-5 * t2 * T);
  return 0.1e1 / (0.1e1 + t7);
}
=#

mf_magma (generic function with 1 method)

In [18]:
function average(mfl, T, C, nl, nx, ny)
    ixl = blockIdx().x * blockDim().x + threadIdx().x
    iyl = blockIdx().y * blockDim().y + threadIdx().y

    if (ixl > (nx / nl - 1) || iyl > (ny / nl - 1))
        return
    end

    avg = 0.0

    #for (int ix = ixl * nl; ix < (ixl + 1) * nl; ++ix) 
    for ix = (ixl*nl):((ixl+1)*nl)

        if (ix > nx - 1)
            break
        end

        #for (int iy = iyl * nl; iy < (iyl + 1) * nl; ++iy) 
        for iy = (iyl*nl):(iy<(iyl+1)*nl)
            if (iy > ny - 1)
                break
            end
            vf = C[idc(ix, iy, nx)]
            avg +=
                mf_magma(T[idc(ix, iy, nx)]) * vf + mf_rock(T[idc(ix, iy, nx)]) * (1 - vf)
            #=  
            avg += mf_magma(T[idc(ix, iy, nx)]) * vf + mf_rock(T[idc(ix, iy, nx)]) * (1 - vf);
            =#
        end
    end

    avg /= nl * nl
    mfl[iyl*(nx/nl)+ixl] = avg

end



#=
__global__ void average(double *mfl, double *T, double *C, int nl, int nx, int ny) {
  int ixl = blockIdx.x * blockDim.x + threadIdx.x;
  int iyl = blockIdx.y * blockDim.y + threadIdx.y;

  if (ixl > (nx / nl - 1) || iyl > (ny / nl - 1)) {
    return;
  }

  double avg = 0.0;
  for (int ix = ixl * nl; ix < (ixl + 1) * nl; ++ix) {
    if (ix > nx - 1) {
      break;
    }
    for (int iy = iyl * nl; iy < (iyl + 1) * nl; ++iy) {
      if (iy > ny - 1) {
        break;
      }
      double vf = C[idc(ix, iy)];
      avg += mf_magma(T[idc(ix, iy)]) * vf + mf_rock(T[idc(ix, iy)]) * (1 - vf);
    }
  }
  avg /= nl * nl;
  mfl[iyl * (nx / nl) + ixl] = avg;
}
=#

average (generic function with 1 method)

In [19]:
using JupyterFormatter
#enable_autoformat()
disable_autoformat()

LoadError: MethodError: no method matching iterate(::Nothing)

[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:880[24m[39m
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Integer[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:880[24m[39m
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}}
[0m[90m   @[39m [90mBase[39m [90m[4mdict.jl:698[24m[39m
[0m  ...


In [1]:
import Random



function main()


    Random.seed!(1234)

    #Настроить фильтр

    #Настроить девайс если не выбран

    #Вывести свойства
    print_gpu_properties()

    dpa = Array{Float64,1}(undef, 18)
    ipa = Array{Int32,1}(undef, 12)

    Lx = 0.0
    Ly = 0.0
    lam_r_rhoCp = 0.0
    lam_m_rhoCp = 0.0
    L_Cp = 0.0
    T_top = 0.0
    T_bot = 0.0
    T_magma = 0.0
    tsh = 0.0
    gamma = 0.0
    Ly_eruption = 0.0
    nu = 0.0
    G = 0.0
    dt = 0.0
    dx = 0.0
    dy = 0.0
    eiter = 0.0
    pic_amount = 0.0

    pmlt = 0
    nx = 0
    ny = 0
    nl = 0
    nt = 0
    niter = 0
    nout = 0
    nsub = 0
    nerupt = 0
    npartcl = 0
    nmarker = 0
    nSample = 0

    #char filename[1024];
    filename = Array{Char,1}(undef, 1024)

    io = open("pa.bin", "r")
    read!(io, dpa)
    read!(io, ipa)


    ipar = 1
    Lx, ipar = read_par(dpa, ipar)
    Ly, ipar = read_par(dpa, ipar)
    lam_r_rhoCp, ipar = read_par(dpa, ipar)
    lam_m_rhoCp, ipar = read_par(dpa, ipar)
    L_Cp, ipar = read_par(dpa, ipar)
    T_top, ipar = read_par(dpa, ipar)
    T_bot, ipar = read_par(dpa, ipar)
    T_magma, ipar = read_par(dpa, ipar)
    tsh, ipar = read_par(dpa, ipar)
    gamma, ipar = read_par(dpa, ipar)
    Ly_eruption, ipar = read_par(dpa, ipar)
    nu, ipar = read_par(dpa, ipar)
    G, ipar = read_par(dpa, ipar)
    dt, ipar = read_par(dpa, ipar)
    dx, ipar = read_par(dpa, ipar)
    dy, ipar = read_par(dpa, ipar)
    eiter, ipar = read_par(dpa, ipar)
    pic_amount, ipar = read_par(dpa, ipar)

    ipar = 1

    pmlt, ipar = read_par(ipa, ipar)
    nx, ipar = read_par(ipa, ipar)
    ny, ipar = read_par(ipa, ipar)
    nl, ipar = read_par(ipa, ipar)
    nt, ipar = read_par(ipa, ipar)
    niter, ipar = read_par(ipa, ipar)
    nout, ipar = read_par(ipa, ipar)
    nsub, ipar = read_par(ipa, ipar)
    nerupt, ipar = read_par(ipa, ipar)
    npartcl, ipar = read_par(ipa, ipar)
    nmarker, ipar = read_par(ipa, ipar)
    nSample, ipar = read_par(ipa, ipar)

    critVol = Array{Float64,1}(undef, nSample)
    read!(io, critVol)

    ndikes = Array{Int32,1}(undef, nt)
    read!(io, ndikes)

    ndikes_all = 0

    for istep = 1:nt
        ndikes_all = ndikes_all + ndikes[istep]
    end

    particle_edges = Array{Int32,1}(undef, ndikes_all + 1)
    read!(io, particle_edges)

    marker_edges = Array{Int32,1}(undef, ndikes_all + 1)
    read!(io, marker_edges)

    close(io)

    cap_frac = 1.5
    npartcl0 = npartcl
    max_npartcl = convert(Int64, npartcl * cap_frac) + particle_edges[ndikes_all]


    nmarker0 = nmarker

    max_nmarker = nmarker + marker_edges[ndikes_all]


    #dim3 blockSize(16, 32);
    # dim3 gridSize((nx + blockSize.x - 1) / blockSize.x, (ny + blockSize.y - 1) / blockSize.y);

    T = CuArray{Float64,2}(undef, nx, ny)
    T_old = CuArray{Float64,2}(undef, nx, ny)
    C = CuArray{Float64,2}(undef, nx, ny)
    wts = CuArray{Float64,2}(undef, nx, ny)
    pcnt = CuArray{Int32,2}(undef, nx, ny)

    a = CuArray{Float64}(undef, (1, 2))

    px = CuArray{Float64}(undef, max_npartcl)
    py = CuArray{Float64}(undef, max_npartcl)
    pT = CuArray{Float64}(undef, max_npartcl)
    pPh = CuArray{Int8}(undef, max_npartcl)

    np_dikes = particle_edges[ndikes_all]

    px_dikes = CuArray{Float64,1}(undef, np_dikes)
    py_dikes = CuArray{Float64,1}(undef, np_dikes)

    mx = CuArray{Float64,1}(undef, max_nmarker)
    my = CuArray{Float64,1}(undef, max_nmarker)
    mT = CuArray{Float64,1}(undef, max_nmarker)

    staging = Array{Float64,1}(undef, max_npartcl)
    npartcl_d = Array{Int32,1}(undef, 1)


    nxl = convert(Int64, nx / nl)
    nyl = convert(Int64, ny / nl)

    L = CuArray{Int32,2}(undef, nxl, nyl)

    L_host = Array{Int32,2}(undef, nxl, nyl)

    mfl = CuArray{Float64,2}(undef, nxl, nyl)

    dike_a = Array{Float64,1}(undef, ndikes_all)
    dike_b = Array{Float64,1}(undef, ndikes_all)
    dike_x = Array{Float64,1}(undef, ndikes_all)
    dike_y = Array{Float64,1}(undef, ndikes_all)
    dike_t = Array{Float64,1}(undef, ndikes_all)

    #=
    io = open("dikes.bin", "r");
    read!(io, dike_a)
    read!(io, dike_b)
    read!(io, dike_x)
    read!(io, dike_y)
    read!(io, dike_t)

    close(io)

    fid = h5open("particles.h5", "r")
    px = read(fid,"px")
    py = read(fid,"py")
    px_dikes = read(fid,"px_dikes")
    py_dikes = read(fid,"py_dikes")
    close(fid)

    fid = h5open("markers.h5", "r")

    obj = fid["0"]
    read(obj, "mx")
    read(obj, "my")
    read(obj, "mT")

    close(fid)
    =#

    NDIGITS = 4

    filename = "grid." * "0"^NDIGITS * "0" * ".h5"

    fid = h5open(filename, "r")
    T_h = read(fid, "T")
    copyto!(T_h, T)
    C_h = read(fid, "C")
    copyto!(C_h, C)
    close(fid)

    #auto tm_all = tic();

    bar1 = "├──"
    bar2 = "\t ├──"
    #bar2 = "\xb3  \xc3\xc4\xc4";

    @time begin
        #init
        @time begin
            @printf("%s initialization              ", bar1)
            pic_amount_tmp = pic_amount
            pic_amount = 1.0

            blockSize1D = 768
            gridSize1D = convert(Int64, floor((npartcl + blockSize1D - 1) / blockSize1D))


            #gridSize1D

            #@device_code_warntype


            #@cuda blocks = gridSize1D threads=blockSize1D g2p(@ALL_ARGS())
            #kekw = idc(2, 4, nx)
            #println("$kekw");

            #@cuda blocks = gridSize1D threads=blockSize1D g2p!(T, T_old, C, wts, px, py, pT, pPh, lam_r_rhoCp, lam_m_rhoCp, L_Cp, T_top, T_bot, dx, dy, dt, pic_amount, nx, ny, npartcl, npartcl0)

            gridSize1D = convert(
                Int64,
                floor((max_npartcl - npartcl + blockSize1D - 1) / blockSize1D),
            )

            #@cuda blocks = gridSize1D threads=blockSize1D init_particles_T(pT, T_magma, max_npartcl, npartcl);
            #@cuda blocks = gridSize1D threads=blockSize1D init_particles_T(pPh, 1, max_npartcl, npartcl);  
            gridSize1D = (max_nmarker - nmarker + blockSize1D - 1) / blockSize1D
            #@cuda blocks = gridSize1D threads=blockSize1D init_particles_T(mT, T_magma, max_nmarker, nmarker); 
            synchronize()

            pic_amount = pic_amount_tmp
        end

        idike = 1
        iSample = 1

        eruptionSteps = Vector{Int32}()

        for it = 1:nt
            #action
            @printf("\n%s it = %d", bar1, it)
            is_eruption = false
            is_intrusion = (ndikes[it] > 0)

            if (it % nerupt == 0)
                @time begin
                    @printf("\n%s checking melt fraction   | ", bar2)

                    blockSizel = (16, 32)
                    gridSizel = ((nxl + blockSizel[1] - 1) / blockSizel[1], (nyl + blockSizel[2] - 1) / blockSizel[2]);
                    
                    @cuda blocks = gridSizel[1], gridSizel[2] threads=blockSizel[1], blockSizel[2] average(mfl, T, C, nl, nx, ny);
                    #average<<<gridSizel, blockSizel>>>(mfl, T, C, nl, nx, ny);
                    #=
                    CUCHECK(cudaDeviceSynchronize());
                    ccl(mfl, L, tsh, nxl, nyl);
                    cudaMemcpy(L_host, L, SIZE_2D(nxl, nyl, int), cudaMemcpyDeviceToHost);

                    std::unordered_map<int, int> volumes;

                    for (int iy = 0; iy < nyl; ++iy) {
                    if (iy * dy * nl < Ly_eruption) {
                    continue;
                    }
                    for (int ix = 0; ix < nxl; ++ix) {
                    if (L_host[iy * nxl + ix] >= 0) {
                    volumes[L_host[iy * nxl + ix]]++;
                    }
                    }
                    }
                    =#
                    maxVol = -1
                    maxIdx = -1
                    #=
                          for (const auto &[idx, vol] : volumes) {
                            if (vol > maxVol) {
                              maxVol = vol;
                              maxIdx = idx;
                            }
                          }
                    =#

                end

                dxl = dx * nl
                dyl = dy * nl

                if (maxVol * dxl * dyl >= critVol[iSample])
                #if (true)
                    @printf("%s erupting %07d cells   | ", bar2, maxVol)
                    @time begin
                        #=
                        auto tm = tic();
                        int *cell_idx;
                        cudaMalloc(&cell_idx, SIZE_1D(maxVol, int));
                        int *cell_idx_host = (int *)malloc(SIZE_1D(maxVol, int));

                        int next_idx = 0;
                        for (int idx = 0; idx < nxl * nyl; ++idx) {
                        if (L_host[idx] == maxIdx) {
                        cell_idx_host[next_idx++] = idx;
                        }
                        }

                        cudaMemcpy(cell_idx, cell_idx_host, SIZE_1D(maxVol, int), cudaMemcpyHostToDevice);

                        blockSize1D.x = 512;
                        gridSize1D.x = (npartcl + blockSize1D.x - 1) / blockSize1D.x;
                        advect_particles_eruption<<<gridSize1D, blockSize1D>>>(px, py, cell_idx, gamma, dxl, dyl, npartcl, maxVol, nxl, nyl);
                        CUCHECK(cudaDeviceSynchronize());
                        gridSize1D.x = (nmarker + blockSize1D.x - 1) / blockSize1D.x;
                        advect_particles_eruption<<<gridSize1D, blockSize1D>>>(mx, my, cell_idx, gamma, dxl, dyl, nmarker, maxVol, nxl, nyl);
                        CUCHECK(cudaDeviceSynchronize());

                        cudaFree(cell_idx);
                        free(cell_idx_host);

                        iSample++;

                        is_eruption = true;
                        eruptionSteps.push_back(it);

                        toc(tm);
                        }   

                        =#
                    end
                end
            end

            if (is_intrusion)
                @printf("%s inserting %02d dikes       | ", bar2, ndikes[it])
                @time begin
                    #=
                    auto tm = tic();

                    for (int i = 0; i < ndikes[it - 1]; ++i, ++idike) {
                    blockSize1D.x = 512;
                    // advect particles
                    gridSize1D.x = (npartcl + blockSize1D.x - 1) / blockSize1D.x;
                    advect_particles_intrusion<<<gridSize1D, blockSize1D>>>(px, py, dike_a[idike], dike_b[idike], dike_x[idike], dike_y[idike], dike_t[idike], nu, G,
                                                        ndikes[it - 1], npartcl);
                    int dike_start = particle_edges[idike];
                    int dike_end = particle_edges[idike + 1];
                    int np_dike = dike_end - dike_start;

                    if (npartcl + np_dike > max_npartcl) {
                    fprintf(stderr, "ERROR: number of particles exceeds maximum value, increase capacity\n");
                    exit(EXIT_FAILURE);
                    }

                    cudaMemcpy(&px[npartcl], &px_dikes[dike_start], np_dike * sizeof(double), cudaMemcpyDeviceToDevice);
                    cudaMemcpy(&py[npartcl], &py_dikes[dike_start], np_dike * sizeof(double), cudaMemcpyDeviceToDevice);
                    npartcl += np_dike;
                    // advect markers
                    gridSize1D.x = (nmarker + blockSize1D.x - 1) / blockSize1D.x;
                    advect_particles_intrusion<<<gridSize1D, blockSize1D>>>(mx, my, dike_a[idike], dike_b[idike], dike_x[idike], dike_y[idike], dike_t[idike], nu, G,
                                                        ndikes[it - 1], nmarker);
                    CUCHECK(cudaDeviceSynchronize());
                    nmarker += marker_edges[idike + 1] - marker_edges[idike];
                    }
                    toc(tm);
                    =#
                end
            end



            if (is_eruption || is_intrusion)
                @printf("%s p2g interpolation        | ", bar2)
                @time begin
                    #=tm = tic();

                    cudaMemset(T, 0, SIZE_2D(nx, ny, double));
                    cudaMemset(C, 0, SIZE_2D(nx, ny, double));
                    cudaMemset(wts, 0, SIZE_2D(nx, ny, double));

                    blockSize1D.x = 768;
                    gridSize1D.x = (npartcl + blockSize1D.x - 1) / blockSize1D.x;
                    p2g_project<<<gridSize1D, blockSize1D>>>(ALL_ARGS);
                    CUCHECK(cudaDeviceSynchronize());
                    p2g_weight<<<gridSize, blockSize>>>(ALL_ARGS);
                    CUCHECK(cudaDeviceSynchronize());
                    toc(tm);

                      =#
                end
                @printf("%s particle injection        | ", bar2)
                @time begin
                    #tm = tic();
                    #=
                    blockSize1D.x = 512;
                    gridSize1D.x = (npartcl + blockSize1D.x - 1) / blockSize1D.x;
                    cudaMemset(pcnt, 0, SIZE_2D(nx, ny, int));
                    count_particles<<<gridSize1D, blockSize1D>>>(pcnt, px, py, dx, dy, nx, ny, npartcl);
                    CUCHECK(cudaDeviceSynchronize());

                    cudaMemcpy(npartcl_d, &npartcl, sizeof(int), cudaMemcpyHostToDevice);
                    int min_pcount = 2;
                    inject_particles<<<gridSize, blockSize>>>(px, py, pT, pPh, npartcl_d, pcnt, T, C, dx, dy, nx, ny, min_pcount, max_npartcl);
                    CUCHECK(cudaDeviceSynchronize());
                    int new_npartcl = npartcl;
                    cudaMemcpy(&new_npartcl, npartcl_d, sizeof(int), cudaMemcpyDeviceToHost);
                    if (new_npartcl > max_npartcl) {
                    fprintf(stderr, "ERROR: number of particles exceeds maximum value, increase capacity\n");
                    exit(EXIT_FAILURE);
                    }
                    if (new_npartcl > npartcl) {
                    printf("(%03d) \xb3 ", new_npartcl - npartcl);
                    npartcl = new_npartcl;
                    } else {
                    printf("(000) \xb3 ", bar2);
                    }
                    end
                    #toc(tm);
                    =#
                end
            end




            @time begin
                @printf("%s solving heat diffusion   | ", bar2)
                #=
                tm = tic();
                CUCHECK(cudaMemcpy(T_old, T, SIZE_2D(nx, ny, double), cudaMemcpyDeviceToDevice));
                for (int isub = 0; isub < nsub; ++isub) {
                update_T<<<gridSize, blockSize>>>(ALL_ARGS);
                CUCHECK(cudaDeviceSynchronize());
                }
                toc(tm);
                =#
            end

            @time begin
                @printf("%s g2p interpolation        | ", bar2)
                #tm = tic();
                #=
                // particles g2p
                blockSize1D.x = 1024;
                gridSize1D.x = (npartcl + blockSize1D.x - 1) / blockSize1D.x;
                g2p<<<gridSize1D, blockSize1D>>>(ALL_ARGS);
                // markers g2p
                gridSize1D.x = (nmarker + blockSize1D.x - 1) / blockSize1D.x;
                double pic_amount_tmp = pic_amount;
                pic_amount = 1.0;
                g2p<<<gridSize1D, blockSize1D>>>(T, T_old, C, wts, mx, my, mT, NULL, lam_r_rhoCp, lam_m_rhoCp, L_Cp, T_top, T_bot, dx, dy, dt, pic_amount, nx, ny, nmarker,
                             nmarker0);
                CUCHECK(cudaDeviceSynchronize());
                pic_amount = pic_amount_tmp;
                toc(tm); 
                =#
            end

            if (it % nout == 0 || is_eruption)
                #auto tm = tic();
                @time begin
                    @printf("%s writing results to disk  | ", bar2)
                    #=
                    sprintf(filename, "grid.%0*d.h5", NDIGITS, it);
                    hsize_t dims[] = {(hsize_t)ny, (hsize_t)nx};
                    hsize_t chunks[] = {(hsize_t)ny, (hsize_t)nx};
                    fid = H5Fcreate(filename, H5F_ACC_TRUNC, H5P_DEFAULT, H5P_DEFAULT);
                    write_h5d(fid, T, SIZE_2D(nx, ny, double), dims, chunks, staging);
                    write_h5d(fid, C, SIZE_2D(nx, ny, double), dims, chunks, staging);
                    if (is_eruption) {
                    hsize_t dims[] = {(hsize_t)nyl, (hsize_t)nxl};
                    hsize_t chunks[] = {(hsize_t)nyl, (hsize_t)nxl};
                    write_h5i(fid, L, SIZE_2D(nxl, nyl, int), dims, chunks, staging);
                    }
                    H5Fclose(fid);
                    toc(tm);
                    =#
                end
            end


            #=
            auto tm = tic();
            printf("%s writing markers to disk  \xb3 ", bar2);
            fid = H5Fopen("markers.h5", H5F_ACC_RDWR, H5P_DEFAULT);
            hsize_t dims1d[] = {(hsize_t)nmarker};
            hsize_t chunks1d[] = {(hsize_t)nmarker};
            char _gname[1024];
            sprintf(_gname, "%d", it);
            hid_t gid = H5Gcreate(fid, _gname, H5P_DEFAULT, H5P_DEFAULT, H5P_DEFAULT);
            write_h5d(gid, mx, SIZE_1D(nmarker, double), dims1d, chunks1d, staging);
            write_h5d(gid, my, SIZE_1D(nmarker, double), dims1d, chunks1d, staging);
            write_h5d(gid, mT, SIZE_1D(nmarker, double), dims1d, chunks1d, staging);
            H5Gclose(gid);
            H5Fclose(fid);
            toc(tm);   
            =#




        end

        @printf("\nTotal time: ")
    end #for_time
    #=   
     printf("Total time: ");
      toc(tm_all);

      h = fopen("eruptions.bin", "wb");
      fwrite(&iSample, sizeof(int), 1, h);
      fwrite(eruptionSteps.data(), sizeof(int), iSample, h);
      fclose(h);

      cudaFree(T);
      cudaFree(T_old);
      cudaFree(C);
      cudaFree(px);
      cudaFree(py);
      cudaFree(pT);
      cudaFree(mx);
      cudaFree(my);
      cudaFree(mT);
      cudaFree(mfl);
      cudaFree(L);
      cudaFreeHost(staging);
      cudaFree(npartcl_d);

      free(critVol);
      free(L_host);
      free(dike_a);
      free(dike_b);
      free(dike_x);
      free(dike_y);
      free(dike_t);
      free(particle_edges);
      free(marker_edges);    
    =#

end

main()

LoadError: LoadError: UndefVarError: `@printf` not defined
in expression starting at In[1]:216